In [1]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from qamsi.config.trading_config import TradingConfig
from qamsi.runner import Runner
from qamsi.strategies.estimated.min_var import MinVariance
from qamsi.cov_estimators.cov_estimators import CovEstimators
from qamsi.features.preprocessor import Preprocessor
from run import Dataset

(CVXPY) Jun 11 08:03:18 PM: Encountered unexpected exception importing solver PROXQP:
ImportError("dlopen(/Users/buchkovv/qamsi/.venv/lib/python3.12/site-packages/cmeel.prefix/lib/python3.12/site-packages/proxsuite/proxsuite_pywrap.cpython-312-darwin.so, 0x0002): Library not loaded: @rpath/libc++.1.dylib\n  Referenced from: <73C5C23C-530A-3E0E-A88E-7897A0C69618> /Users/buchkovv/qamsi/.venv/lib/python3.12/site-packages/cmeel.prefix/lib/python3.12/site-packages/proxsuite/proxsuite_pywrap.cpython-312-darwin.so\n  Reason: tried: '/Users/runner/miniconda3/envs/proxsuite/lib/libc++.1.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/runner/miniconda3/envs/proxsuite/lib/libc++.1.dylib' (no such file), '/var/folders/0j/bwqcs4y508s2n4ck4dhf3rpc0000gn/T/cmeel-qko5evau/whl/cmeel.prefix/lib/libc++.1.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/var/folders/0j/bwqcs4y508s2n4ck4dhf3rpc0000gn/T/cmeel-qko5evau/whl/cmeel.prefix/lib/libc++.1.dylib' (no such file), '/Users

In [3]:
REBAL_FREQ = "ME"
DATASET = Dataset.SPX_US
ESTIMATION_WINDOW = 365 * 1

In [4]:
experiment_config = DATASET.value()

stocks = tuple(
    pd.read_csv(experiment_config.PATH_OUTPUT / experiment_config.STOCKS_LIST_FILENAME)
    .iloc[:, 0]
    .astype(str)
    .tolist(),
)
experiment_config.ASSET_UNIVERSE = stocks  # type: ignore  # noqa: PGH003

experiment_config.MIN_ROLLING_PERIODS = ESTIMATION_WINDOW + 1
experiment_config.N_LOOKBEHIND_PERIODS = None
experiment_config.REBALANCE_FREQ = REBAL_FREQ

factors = pd.read_csv(experiment_config.PATH_OUTPUT / "factors.csv")
factors["date"] = pd.to_datetime(factors["date"])
factors = factors.set_index("date")
factor_names = tuple(factors.columns.astype(str).tolist())
experiment_config.FACTORS = factor_names

prices = [stock + "_Price" for stock in list(stocks)]
preprocessor = Preprocessor(
    exclude_names=[
        *list(stocks),
        experiment_config.RF_NAME,
        *experiment_config.HEDGING_ASSETS,
        *factor_names,
        *prices,
    ],
)

trading_config = TradingConfig(
    broker_fee=0.05 / 100,
    bid_ask_spread=0.03 / 100,
    total_exposure=1,
    max_exposure=1,
    min_exposure=0,
    trading_lag_days=1,
)

runner = Runner(
    experiment_config=experiment_config,
    trading_config=trading_config,
    verbose=False,
)

In [16]:
import numpy as np

class ShrinkageGoldenSection:
    GS_CONST = (1 + np.sqrt(5)) / 2

    def __init__(
        self,
        experiement_runner: Runner,
        start: pd.Timestamp,
        end: pd.Timestamp,
        max_shrinkage: float = 1,
        max_iterations: int = 10,
        acq_beta: float = 1.96,
    ) -> None:
        self.runner = experiement_runner
        self.start = start
        self.end = end
        self.acq_beta = acq_beta
        self._domain_ra = (0, max_shrinkage)
        self.max_iterations = max_iterations

        self.previous_points = []
        self.previous_targets = []

        self.optimal_shrinkage = None

    def optimize(self, ra: float) -> float:
        estimator = CovEstimators.RISKFOLIO.value(
            alpha=ra,
            estimator_type="shrunk",
        )

        strategy = MinVariance(
            cov_estimator=estimator,
            trading_config=trading_config,
            window_size=ESTIMATION_WINDOW,
        )

        fitted_r = runner.run_one_step(
            start_date=self.start,
            end_date=self.end,
            feature_processor=preprocessor,
            strategy=strategy,
        )
        return -fitted_r.std().item()

    def solve(self) -> float:
        return self._run_search(self._domain_ra[1], self.max_iterations)

    def __call__(self):
        return self.solve()

    def _run_search(self, max_ra: float, max_iter: int) -> float:
            min_ra = 0  # Risk-aversion is always non-negative

            left_ra = min_ra + (max_ra - min_ra) / (self.GS_CONST + 1)
            right_ra = max_ra - (max_ra - min_ra) / (self.GS_CONST + 1)
            left_sr = self.optimize(left_ra)
            right_sr = self.optimize(right_ra)
            for _ in range(max_iter):
                if left_sr > right_sr:
                    max_ra = right_ra
                    right_ra = left_ra
                    right_sr = left_sr
                    left_ra = min_ra + (max_ra - min_ra) / (self.GS_CONST + 1)
                    left_sr = self.optimize(left_ra)
                else:
                    min_ra = left_ra
                    left_ra = right_ra
                    left_sr = right_sr
                    right_ra = max_ra - (max_ra - min_ra) / (self.GS_CONST + 1)
                    right_sr = self.optimize(right_ra)

            if left_sr <= right_sr:
                opt_ra = right_ra
                opt_sharpe = right_sr
            else:
                opt_ra = left_ra
                opt_sharpe = left_sr

            self.optimal_sharpe = opt_sharpe

            return opt_ra

In [17]:
start_date = pd.Timestamp("2016-01-01")
end_date = runner.returns.simple_returns.loc[start_date:].iloc[20:].index[0]

start_date, end_date

(Timestamp('2016-01-01 00:00:00'), Timestamp('2016-02-02 00:00:00'))

In [18]:
opt = ShrinkageGoldenSection(
    experiement_runner=runner,
    start=start_date,
    end=end_date,
)
opt()

np.float64(0.029416855007991537)

In [19]:
-opt.optimal_sharpe, -opt.optimize(0.1)

(0.007124307545936087, 0.007195512816994873)

In [ ]:
import gc

available_dates = runner.returns.simple_returns.index

optimal = []
for date in available_dates[251:]:
    start_date = date
    end_date = runner.returns.simple_returns.loc[start_date:].iloc[20:].index[0]

    opt = ShrinkageGoldenSection(
        experiement_runner=runner,
        start=start_date,
        end=end_date,
    )
    opt_ra = opt()

    print(f"Date: {start_date}, Volatility: {-opt.optimal_sharpe * np.sqrt(252):.6f}, Naive Volatility: {-opt.optimize(0.1) * np.sqrt(252):.6f}, Shrinkage: {opt_ra:.2f}")

    optimal.append([start_date, opt_ra])
    gc.collect()

Date: 2005-03-18 00:00:00, Volatility: 0.094022, Naive Volatility: 0.096129, Shrinkage: 0.32
Date: 2005-03-21 00:00:00, Volatility: 0.093738, Naive Volatility: 0.095455, Shrinkage: 0.25
Date: 2005-03-22 00:00:00, Volatility: 0.096399, Naive Volatility: 0.097999, Shrinkage: 0.24
Date: 2005-03-23 00:00:00, Volatility: 0.115121, Naive Volatility: 0.124503, Shrinkage: 0.49
Date: 2005-03-24 00:00:00, Volatility: 0.117243, Naive Volatility: 0.126993, Shrinkage: 0.49
Date: 2005-03-28 00:00:00, Volatility: 0.120212, Naive Volatility: 0.128558, Shrinkage: 0.48
Date: 2005-03-29 00:00:00, Volatility: 0.121938, Naive Volatility: 0.127735, Shrinkage: 0.40
Date: 2005-03-30 00:00:00, Volatility: 0.117597, Naive Volatility: 0.124989, Shrinkage: 0.46
Date: 2005-03-31 00:00:00, Volatility: 0.121307, Naive Volatility: 0.131892, Shrinkage: 0.55
Date: 2005-04-01 00:00:00, Volatility: 0.126590, Naive Volatility: 0.135888, Shrinkage: 0.63
Date: 2005-04-04 00:00:00, Volatility: 0.127042, Naive Volatility: 0.1